# Autoencoder variacional - Treino

# 1 - O problema com os autoencoders tradicionais

## Como geramos novos pontos?

## **A ideia:** Se o espaço latente (imagem do encoder) seguir uma distribuição regular o suficiente, podemos aleatoriamente escolher um ponto x no espaço latente e, com  D(x), gerar a imagem de um novo dígito.

## **O problema:** Não podemos garantir uma distribuição conveniente do espaço latente.

![title](img/AE_Problem.jpg)

## 1. A distribuição não é limitada ou simétrica. Ou seja, é desconhecida.

## 2. A área de representação dos dígitos varia muito.

## 3. Grandes espaços vazios.

# 2 - A solução

## 2.1 - A normal multivariada

## 2.2 - Mapeamento dos pontos a uma distribuição

![title](img/NormalDistrib.jpg)

![title](img/VariationalDistri.jpg)

## 2.3 - O que significa "sample de uma distribuição"

![title](img/car_colors.gif)

# 3 - Regularização

## - Resolvemos o problema 1. Mas como garantir que os valores da média e variância sejam razoáveis?

## - Queremos: completude e continuidade

## **Solução:** Alterar a Loss Function

## Divergência de Kullback-Leibler

![title](img/KLLoss.png)

![title](img/WithoutWithReg.png)

![title](img/MorphGrad.png)

## Loss function = r_loss_factor * r_loss + kl_divergence

## **Tradeoff do r_loss_factor:**

## - Se for muito grande, recaímos nos problemas do autoencoder inicial.

## - Se for muito pequeno, o espaço latente será regular, porém irá gerar reconstruções de baixa qualidade.

# 4 - O código

In [13]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

In [14]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

In [15]:
(x_train, y_train), (x_test, y_test) = load_mnist()

# 5 - Criação da Rede Neural

In [16]:
from tensorflow.python.framework.ops import disable_eager_execution

vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [17]:
vae.encoder.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_7[0][0]              
____________________________________________________________________________________________

In [18]:
vae.decoder.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 14, 14, 64)        0   

# 6 - Treino da Rede Neural

In [19]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [20]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [21]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [22]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Train on 60000 samples
Epoch 1/200
   96/60000 [..............................] - ETA: 3:47 - loss: 230.4904 - vae_r_loss: 230.4890 - vae_kl_loss: 0.0015    

C:\Users\Pedro\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


60000/60000 [==============================] - ETA: 0s - loss: 58.8257 - vae_r_loss: 55.6236 - vae_kl_loss: 3.2020
Epoch 00001: saving model to run/vae/0002_digits\weights\weights-001-58.83.h5

Epoch 00001: saving model to run/vae/0002_digits\weights\weights.h5
60000/60000 [==============================] - 68s 1ms/sample - loss: 58.8257 - vae_r_loss: 55.6236 - vae_kl_loss: 3.2020
Epoch 2/200
59968/60000 [============================>.] - ETA: 0s - loss: 51.9720 - vae_r_loss: 48.0950 - vae_kl_loss: 3.8770
Epoch 00002: saving model to run/vae/0002_digits\weights\weights-002-51.97.h5

Epoch 00002: saving model to run/vae/0002_digits\weights\weights.h5
60000/60000 [==============================] - 66s 1ms/sample - loss: 51.9722 - vae_r_loss: 48.0952 - vae_kl_loss: 3.8770
Epoch 3/200
60000/60000 [==============================] - ETA: 0s - loss: 50.2820 - vae_r_loss: 46.0787 - vae_kl_loss: 4.2033
Epoch 00003: saving model to run/vae/0002_digits\weights\weights-003-50.28.h5

Epoch 00003: s